In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools
import numpy as np
import mechanicalsoup

In [2]:
# Login to Godville and navigate to the crossword page
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://godvillegame.com/")
browser.select_form('form[action="/login/login"]')
browser["username"] = "W2t2UcMEYRUDCav"
browser["password"] = "W2t2UcMEYRUDCav"
response = browser.submit_selected()
browser.follow_link("news")

<Response [200]>

In [3]:
browser.get_url()

'https://godvillegame.com/news'

In [4]:
soup = browser.get_current_page()

html_doc = ""
#soup = BeautifulSoup(html_doc, 'html.parser')

# print(soup.prettify())

In [5]:
# Initialize variables to store crossword information
across_comingup = 0
down_comingup = 0
word_index_across = []
word_index_down = []
def read_cross_q(word_index):
    """
    Read the crossword questions from the crossword page and store the corresponding word indices.

    Args:
    word_index (list): Empty list to store word indices.
    """
    splitted = line.split()
    for _ in splitted:
        if _.isdigit():
            word_index.append(int(_))
        else:
            pass

# Parse the crossword page HTML and extract the word indices and positions of unfilled letters
for string in soup.find("div", attrs={"class": "cross_q"}).stripped_strings:
    line = repr(string)
    line = line.replace('\\t', '')
    line = line.replace('\\n', ' ')
    line = line.replace('&nbsp', ' ')
    line = line.replace('.', '')
    line = line.replace('\'', '')
    line = line.replace('\\xa0', ' ')
    if line == "Across:":
        across_comingup = 1
    elif line == "Down:":
        across_comingup = 0
        down_comingup = 1
    elif across_comingup:
        read_cross_q(word_index_across)
    elif down_comingup:
        read_cross_q(word_index_down)


In [6]:
across_words = [[] for i in range(len(word_index_across))]
down_words = [[] for i in range(len(word_index_down))]
list_of_unfilled_letter_positions = [[]]
counter = 0
for unknown_letters in soup.find_all("input", attrs={"autocomplete":"off"}):
    try:
        line = unknown_letters['aria-label']
    except KeyError:
        continue
    line = line.replace(',', '')
    line = line.replace('.', ' .')
    splitted = line.split()
    # print(splitted)
    if (splitted[1] == 'down'):
        down_words[word_index_down.index(int(splitted[0]))].append([])
        list_of_unfilled_letter_positions[counter].append(1)
        list_of_unfilled_letter_positions[counter].append(word_index_down.index(int(splitted[0])))
        list_of_unfilled_letter_positions[counter].append(int(splitted[splitted.index('letter') + 1]))
        list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
    if (splitted[1] == 'across'):
        across_words[word_index_across.index(int(splitted[0]))].append([])
        list_of_unfilled_letter_positions[counter].append(0)
        list_of_unfilled_letter_positions[counter].append(word_index_across.index(int(splitted[0])))
        list_of_unfilled_letter_positions[counter].append(int(splitted[splitted.index('letter') + 1]))
        list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
    if '.' in splitted :
        shifted_index = splitted.index('.') + 1
        counter += 1
        list_of_unfilled_letter_positions.append([])
        if (splitted[shifted_index + 1] == 'down'):
            down_words[word_index_down.index(int(splitted[shifted_index + 0]))].append([])
            list_of_unfilled_letter_positions[counter].append(1)
            list_of_unfilled_letter_positions[counter].append(word_index_down.index(int(splitted[shifted_index + 0])))
            list_of_unfilled_letter_positions[counter].append(int(splitted[splitted[shifted_index:].index('letter') + 1 + shifted_index]))
            list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
        if (splitted[shifted_index + 1] == 'across'):
            across_words[word_index_across.index(int(splitted[shifted_index + 0]))].append([])
            list_of_unfilled_letter_positions[counter].append(0)
            list_of_unfilled_letter_positions[counter].append(word_index_across.index(int(splitted[shifted_index + 0])))
            list_of_unfilled_letter_positions[counter].append(int(splitted[splitted[shifted_index:].index('letter') + 1 + shifted_index]))
            list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
    counter += 1
    list_of_unfilled_letter_positions.append([])
list_of_unfilled_letter_positions.pop()

[]

In [7]:
# Extract the filled letters in the crossword
for unknown_letters in soup.find_all("div", attrs={"class": "open"}):
    line = unknown_letters['aria-label']
    line = line.replace(',', '')
    line = line.replace('.', ' .')
    splitted = line.split()
    # print(splitted)
    if (splitted[1] == 'down'):
        down_words[word_index_down.index(int(splitted[0]))].append([])
    if (splitted[1] == 'across'):
        across_words[word_index_across.index(int(splitted[0]))].append([])
    if '.' in splitted :
        shifted_index = splitted.index('.') + 1
        if (splitted[shifted_index+1] == 'down'):
            down_words[word_index_down.index(int(splitted[0 + shifted_index]))].append([])
        if (splitted[shifted_index+1] == 'across'):
            across_words[word_index_across.index(int(splitted[0 + shifted_index]))].append([])

In [8]:
for unknown_letters in soup.find_all("div", attrs={"class": "open"}):
    line = unknown_letters['aria-label']
    line = line.replace(',', '')
    # line = line.replace('-', '')
    splitted = line.split()
    # print(splitted)
    if splitted[splitted.index("letter") + 1] == splitted[-2]:
        if (splitted[1] == 'down'):
            down_words[word_index_down.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = ' '
        elif (splitted[1] == 'across'):
            across_words[word_index_across.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = ' '
    else:
        if (splitted[1] == 'down'):
            try:
                down_words[word_index_down.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = splitted[-1]
            except ValueError:
                down_words[word_index_down.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1][:-1]) - 1] = splitted[-1]
        elif (splitted[1] == 'across'):
            across_words[word_index_across.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = splitted[-1]

In [9]:
# Get the solution to the crossword from the Omnibus List on the Godville wiki
link_solution = urlopen('https://wiki.godvillegame.com/Omnibus_List').read()    
soup_solution = BeautifulSoup(link_solution, 'html.parser')

#print(soup_solution.prettify())

In [10]:
# Extract the solutions to the crossword from the Omnibus List
across_solution = [[] for i in range(len(word_index_across))]
down_solution = [[] for i in range(len(word_index_down))]
for i in range(len(across_words)):
    l_test = across_words[i]
    for string in soup_solution.find_all("li"):
        line = list(string.text.upper())
        count = np.where([ x == y for x, y in zip(line, l_test) ])[0].size
        if count == len(np.where([x != [] for x in l_test])[0]) and len(l_test) == len(line):
            print("".join(line))
            across_solution[i] = line
for i in range(len(down_words)):
    l_test = down_words[i]
    for string in soup_solution.find_all("li"):
        line = list(string.text.upper())
        count = np.where([ x == y for x, y in zip(line, l_test) ])[0].size
        if count == len(np.where([x != [] for x in l_test])[0]) and len(line) == len(l_test):
            print("".join(line))
            down_solution[i] = line

ASTRAL PROJECTOR
SAD PANDA
CARJACKAL
JERKULES
